In [2]:
import pandas as pd
import numpy as np
import py7zr
import json
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from src import utils

In [3]:
# Load a sample game JSON file
with open("data/json/0021500622.json", "r") as f:
    game = json.load(f)

In [4]:
# Explore the structure of the game data
game.keys()

dict_keys(['gameid', 'gamedate', 'events'])

In [5]:
# 448 "events" in this game
len(game['events'])

448

In [6]:
# Each event has multiple "moments" or frames; for example, the first event has 150 moments
len(game['events'][0]['moments'])

150

In [7]:
# What one moment looks like and the schema
game['events'][0]['moments'][0]

[1,
 1453165380026,
 623.59,
 15.99,
 None,
 [[-1, -1, 28.64068, 45.40327, 3.54135],
  [1610612739, 2544, 6.44329, 22.26184, 0.0],
  [1610612739, 2747, 17.45534, 4.23261, 0.0],
  [1610612739, 201567, 7.51449, 16.49044, 0.0],
  [1610612739, 202681, 27.87412, 45.4898, 0.0],
  [1610612739, 202389, 20.61859, 22.83587, 0.0],
  [1610612744, 101106, 11.4783, 29.89826, 0.0],
  [1610612744, 201939, 22.98616, 42.65101, 0.0],
  [1610612744, 202691, 17.87231, 15.54552, 0.0],
  [1610612744, 203110, 3.52194, 20.92478, 0.0],
  [1610612744, 203084, 8.26079, 24.48449, 0.0]]]

### Schema

| Index | Value           | Meaning                        | Schema Field       |
| ----- | --------------- | ------------------------------ | ------------------ |
| `0`   | `1`             | Quarter                        | `quarter`          |
| `1`   | `1453165380026` | Timestamp (ms since epoch)     | *(not documented)* |
| `2`   | `623.59`        | Game clock (seconds remaining) | `game_clock`       |
| `3`   | `15.99`         | Shot clock (seconds remaining) | `shot_clock`       |
| `4`   | `None`          | Event ID / unused              | *(ignore)*         |
| `5`   | `[...]`         | Ball + player positions        | see below          |


#### Ball Position (always first)
| Field     | Value                   |
| --------- | ----------------------- |
| team_id   | `-1`                    |
| player_id | `-1`                    |
| x         | `28.64`                 |
| y         | `45.40`                 |
| z         | `3.54` (height of ball) |

#### Player Postions (next 10 entries)
| Field    | Value        |
| -------- | ------------ |
| teamid   | `1610612739` |
| playerid | `2544`       |
| x        | `6.44`       |
| y        | `22.26`      |
| z        | `0.0`        |



In [8]:
all_rows = []

for moment in game['events'][0]['moments']:  # unpack the list

    quarter = moment[0]
    timestamp = moment[1]
    game_clock = moment[2]
    shot_clock = moment[3]

    objects = moment[5]
    if len(objects) < 2:
        continue  # skip empty moments

    # first object = ball
    ball = objects[0]
    ball_x, ball_y, ball_z = ball[2], ball[3], ball[4]

    # remaining objects = players
    for p in objects[1:]:
        team_id, player_id, x, y, z = p
        all_rows.append({
            "gameid": game.get('gameid'),
            "quarter": quarter,
            "timestamp": timestamp,
            "game_clock": game_clock,
            "shot_clock": shot_clock,
            "team_id": team_id,
            "player_id": player_id,
            "x": x,
            "y": y,
            "z": z,
            "ball_x": ball_x,
            "ball_y": ball_y,
            "ball_z": ball_z,
            'event_id': 1  # since we're only using the first event
        })

# Convert to DataFrame
df = pd.DataFrame(all_rows)
df.head()

,gameid,quarter,timestamp,game_clock,shot_clock,team_id,player_id,x,y,z,ball_x,ball_y,ball_z,event_id
0,0021500622,1,1453165380026,623.59,15.99,1610612739,2544,6.44329,22.26184,0.0,28.64068,45.40327,3.54135,1
1,0021500622,1,1453165380026,623.59,15.99,1610612739,2747,17.45534,4.23261,0.0,28.64068,45.40327,3.54135,1
2,0021500622,1,1453165380026,623.59,15.99,1610612739,201567,7.51449,16.49044,0.0,28.64068,45.40327,3.54135,1
3,0021500622,1,1453165380026,623.59,15.99,1610612739,202681,27.87412,45.48980,0.0,28.64068,45.40327,3.54135,1
4,0021500622,1,1453165380026,623.59,15.99,1610612739,202389,20.61859,22.83587,0.0,28.64068,45.40327,3.54135,1
